In [14]:
# Challenge: Add a function to rotl that gets citations from a subtree

# First, get a subtree with rotl

res <- rotl::tol_induced_subtree(ott_ids = c(292466, 267845, 316878, 102710))



Warning message in collapse_singles(tr, show_progress):
“Dropping singleton nodes with labels: mrcaott246ott5481, mrcaott246ott5021, mrcaott246ott7145, mrcaott246ott5272, mrcaott246ott928360, mrcaott246ott1858, mrcaott246ott2907, mrcaott246ott3600042, mrcaott246ott47588, mrcaott246ott7113, Passeriformes ott1041547, mrcaott246ott3212, mrcaott246ott428578, mrcaott246ott44866, mrcaott246ott5929, mrcaott246ott32658, mrcaott246ott4820, mrcaott246ott22325, mrcaott246ott176461, mrcaott246ott10351, mrcaott246ott1488, mrcaott246ott3364, mrcaott246ott5934, mrcaott246ott1566, mrcaott1566ott3598440, mrcaott1566ott496009, Cinclidae ott496027, mrcaott4765ott6520193, Galliformes ott837585, mrcaott4765ott6520194, mrcaott4765ott109888, mrcaott4765ott75785, mrcaott4765ott104461, mrcaott4765ott151684, mrcaott4765ott54193, mrcaott4765ott49319, mrcaott4765ott49310, mrcaott4765ott415487, mrcaott4765ott51354, mrcaott4765ott53700, mrcaott53700ott572162, mrcaott53700ott466627, Anseriformes ott241841, mrcaott30

In [15]:
str(res) # this checks the structure of the output file. It is indeed a tree.

List of 4
 $ edge      : int [1:6, 1:2] 5 6 6 7 7 5 6 1 7 2 ...
 $ tip.label : chr [1:4] "Cinclus_ott267845" "Perdix_ott102710" "Clangula_ott316878" "Struthio_ott292466"
 $ Nnode     : num 3
 $ node.label: chr [1:3] "Aves ott81461" "mrcaott246ott4765" "mrcaott4765ott30843"
 - attr(*, "class")= chr "phylo"


In [16]:
ls(res) # with this we can see the contents of the output of tol_induced_subtree 
# it does not return any information on supporting studies for that subtree

[1] "edge"       "Nnode"      "node.label" "tip.label"

In [17]:
rotl::tol_induced_subtree # look at how the function is coded
# try to identify where it is dropping the supporting studies information

function (ott_ids = NULL, node_ids = NULL, label_format = NULL, 
    file, ...) 
{
    res <- .tol_induced_subtree(ott_ids = ott_ids, node_ids = node_ids, 
        label_format = label_format, ...)
    if (!missing(file)) {
        unlink(file)
        cat(res$newick, file = file)
        return(file.exists(file))
    }
    else {
        phy <- phylo_from_otl(res)
        return(phy)
    }
}
<bytecode: 0x7fcf4ec59a58>
<environment: namespace:rotl>

In [20]:
rotl:::.tol_induced_subtree # look at the code of this inner function
# this does not drop the citations

function (ott_ids = NULL, node_ids = NULL, label_format = NULL, 
    ...) 
{
    if (is.null(ott_ids) && is.null(node_ids)) {
        stop("Must provide ", sQuote("ott_ids"), " or ", sQuote("node_ids"), 
            " (or both).")
    }
    q <- list()
    if (!is.null(label_format)) {
        if (!check_label_format(label_format)) {
            stop(sQuote("label_format"), " must be one of: ", 
                sQuote("name"), ", ", sQuote("id"), ", or ", 
                sQuote("name_and_id"))
        }
        q$label_format <- jsonlite::unbox(label_format)
    }
    if (!is.null(ott_ids)) {
        ott_ids <- check_ott_ids(ott_ids)
        q$ott_ids <- ott_ids
    }
    if (!is.null(node_ids)) {
        check_node_ids(node_ids)
        q$node_ids <- node_ids
    }
    if ((length(ott_ids) + length(node_ids)) < 2) {
        stop("At least two valid ", sQuote("ott_ids"), " or ", 
            sQuote("node_ids"), " must be provided.")
    }
    res <- otl_POST("tree_of_life/induced_subtree", body = q, 
        ...)
    res
}
<bytecode: 0x7fcf4ec5ecb8>
<environment: namespace:rotl>

In [23]:
rotl:::otl_POST

function (path, body, url = otl_url(...), otl_v = otl_version(...), 
    ...) 
{
    stopifnot(is.list(body))
    body_json <- ifelse(length(body), jsonlite::toJSON(body), 
        "")
    req <- httr::POST(url, path = paste(otl_v, path, sep = "/"), 
        body = body_json, ...)
    otl_check(req)
}
<bytecode: 0x7fcf4dfe48f0>
<environment: namespace:rotl>

In [22]:
rotl:::phylo_from_otl # this is where citations are dropped
# the function only gets the tree

function (res, dedup = FALSE) 
{
    if (is.list(res)) {
        if (!is.null(res$newick)) {
            tree <- res$newick
        }
        else if (!is.null(res$subtree)) {
            tree <- res$subtree
        }
        else {
            stop("Cannot find tree")
        }
    }
    else if (is.character(res)) {
        tree <- res
    }
    else stop("I don't know how to deal with this format.")
    if (grepl("\\(", tree)) {
        fnm <- tempfile()
        cat(tree, file = fnm)
        if (!dedup) {
            phy <- rncl::read_newick_phylo(fnm)
        }
        else {
            dedup_tr <- deduplicate_labels(fnm)
            phy <- rncl::read_newick_phylo(dedup_tr)
            unlink(dedup_tr)
        }
        unlink(fnm)
    }
    else {
        phy <- tree_to_labels(tree)$tip_label
    }
    return(phy)
}
<bytecode: 0x7fcf4bf755f8>
<environment: namespace:rotl>

In [39]:
?rotl::get_publication
rotl::get_study_meta

Rendering development documentation for 'get_publication'



function(study_id, ...) {
  res <- .get_study_meta(study_id = study_id, ...)
  class(res) <- "study_meta"
  attr(res, "study_id") <- study_id
  res
}
<environment: namespace:rotl>

In [24]:
# create function studies_from_otl

studies_from_otl <- function (res) 
{
    if (is.list(res)) {
        if (!is.null(res$supporting_studies)) {
            studies <- res$supporting_studies
        }
        else {
            stop("Cannot find supporting studies")
        }
    }
#     else if (is.character(res)) {
#         tree <- res
#     }
#     else stop("I don't know how to deal with this format.")
#     if (grepl("\\(", tree)) {
#         fnm <- tempfile()
#         cat(tree, file = fnm)
#         if (!dedup) {
#             phy <- rncl::read_newick_phylo(fnm)
#         }
#         else {
#             dedup_tr <- deduplicate_labels(fnm)
#             phy <- rncl::read_newick_phylo(dedup_tr)
#             unlink(dedup_tr)
#         }
#         unlink(fnm)
#     }
#     else {
#         phy <- tree_to_labels(tree)$tip_label
#     }
    return(studies)
}


In [61]:
# res is an output from .tol_induced_subtree or .tol_subtree
# returns a list with tree and study ids supporting a subtree
studies_from_otl <- function(res){
  if (is.list(res)) {
    if (!is.null(res$supporting_studies)) {
        studies <- res$supporting_studies
    }
    else {
        stop("Cannot find supporting studies")
    }
  }
  study_ids <- sapply(stringr::str_split(studies, "@"), '[', 1)
  trees <- study_idssapply(stringr::str_split(studies, "@"), '[', 2)
  return(list(study_ids, trees))
}

In [40]:
res <- rotl:::.tol_induced_subtree(ott_ids = c(292466, 267845, 316878, 102710))

In [41]:
studies_from_otl(res)

$study_ids
 [1] "ot_425"  "ot_521"  "ot_809"  "pg_2804" "ot_290"  "ot_783"  "pg_2015"
 [8] "ot_531"  "ot_782"  "ot_841"  "pg_2913" "ot_534"  "ot_757"  "pg_420" 
[15] "ot_412"  "ot_500"  "ot_532"  "pg_2404" "pg_2404" "pg_2913" "ot_111" 
[22] "ot_753"  "pg_2577" "ot_1002" "ot_1345" "ot_855"  "ot_1174" "pg_2866"
[29] "ot_1022" "ot_314"  "ot_1516"

$trees
 [1] "tree1"    "tree1"    "tree2"    "tree6511" "tree1"    "tree1"   
 [7] "tree4152" "tree1"    "Tr48736"  "tree1"    "tree6742" "tree1"   
[13] "Tr70438"  "tree522"  "Tr77157"  "tree1"    "tree1"    "tree5068"
[19] "tree6388" "tree6741" "tree1"    "tree1"    "tree5980" "tree1"   
[25] "Tr109614" "tree1"    "tree3"    "tree6656" "tree1"    "tree1"   
[31] "Tr55113"

In [42]:
study_ids_andtrees <- studies_from_otl(res)

In [43]:
class(study_ids_andtrees)

[1] "list"

In [44]:
study_ids_andtrees[[1]]

[1] "ot_425"  "ot_521"  "ot_809"  "pg_2804" "ot_290"  "ot_783"  "pg_2015"
 [8] "ot_531"  "ot_782"  "ot_841"  "pg_2913" "ot_534"  "ot_757"  "pg_420" 
[15] "ot_412"  "ot_500"  "ot_532"  "pg_2404" "pg_2404" "pg_2913" "ot_111" 
[22] "ot_753"  "pg_2577" "ot_1002" "ot_1345" "ot_855"  "ot_1174" "pg_2866"
[29] "ot_1022" "ot_314"  "ot_1516"

In [52]:
write(paste(citations, collapse = '\n'), file = 'cites.txt')

In [1]:
##' @export
##' @param study_ids is a vector of study ids
get_publication.vector <- function(study_id){
  citations <- sapply(study_ids, function(x) rotl::get_publication.study_meta(rotl::get_study_meta(study_id = x)))
  return(citations)
}

In [50]:
getwd()
devtools::load_all('/Users/luna/pj_opentree/rotl')

[1] "/Users/luna/pj_opentree/opentree-ipynb/ipynb"

Loading rotl



# New functions

In [22]:
##' @export
##' @param study_ids is a vector of study ids
get_publication.vector <- function(study_ids){
  citations <- sapply(study_ids, function(x) rotl::get_publication.study_meta(rotl::get_study_meta(study_id = x)))
  return(citations)
}

In [23]:
# res is an output from .tol_induced_subtree or .tol_subtree
# returns a list with tree and study ids supporting a subtree
studies_from_otl <- function(res){
  if (is.list(res)) {
    if (!is.null(res$supporting_studies)) {
        studies <- res$supporting_studies
    }
    else {
        stop("Cannot find supporting studies")
    }
  }
  study_ids <- sapply(stringr::str_split(studies, "@"), '[', 1)
  trees <- sapply(stringr::str_split(studies, "@"), '[', 2)
  return(list(study_ids = study_ids, trees = trees))
}

In [55]:
# add arguments to .tol_induced_subtree
# get_citations = TRUE

tol_induced_subtree <- function(ott_ids = NULL, node_ids = NULL, label_format = NULL,
                                file, get_citations = TRUE, ...) {
  res <- .tol_induced_subtree(
    ott_ids = ott_ids, node_ids = node_ids,
    label_format = label_format, ...
  )
  if(get_citations){
    if (!missing(file)) {
      filecit <- paste0(file, "_citations.txt")
    } else {
      filecit <- "citations.txt"
    }
    citations <- get_publication.vector(studies_from_otl(res)$study_ids)
    unlink(filecit)
    cat(paste(citations, collapse = '\n\n'), file = filecit)
  }
  if (!missing(file)) {
    unlink(file)
    cat(res$newick, file = file)
    return(file.exists(file))
  } else {
    phy <- phylo_from_otl(res)
    return(phy)
  }
}


In [56]:
tol_induced_subtree(c(292466, 267845, 316878, 102710))

Warning message in collapse_singles(tr, show_progress):
“Dropping singleton nodes with labels: mrcaott246ott5481, mrcaott246ott5021, mrcaott246ott7145, mrcaott246ott5272, mrcaott246ott928360, mrcaott246ott1858, mrcaott246ott2907, mrcaott246ott3600042, mrcaott246ott47588, mrcaott246ott7113, Passeriformes ott1041547, mrcaott246ott3212, mrcaott246ott428578, mrcaott246ott44866, mrcaott246ott5929, mrcaott246ott32658, mrcaott246ott4820, mrcaott246ott22325, mrcaott246ott176461, mrcaott246ott10351, mrcaott246ott1488, mrcaott246ott3364, mrcaott246ott5934, mrcaott246ott1566, mrcaott1566ott3598440, mrcaott1566ott496009, Cinclidae ott496027, mrcaott4765ott6520193, Galliformes ott837585, mrcaott4765ott6520194, mrcaott4765ott109888, mrcaott4765ott75785, mrcaott4765ott104461, mrcaott4765ott151684, mrcaott4765ott54193, mrcaott4765ott49319, mrcaott4765ott49310, mrcaott4765ott415487, mrcaott4765ott51354, mrcaott4765ott53700, mrcaott53700ott572162, mrcaott53700ott466627, Anseriformes ott241841, mrcaott30


Phylogenetic tree with 4 tips and 3 internal nodes.

Tip labels:
[1] "Cinclus_ott267845"  "Perdix_ott102710"   "Clangula_ott316878"
[4] "Struthio_ott292466"
Node labels:
[1] "Aves ott81461"       "mrcaott246ott4765"   "mrcaott4765ott30843"

Rooted; no branch lengths.

In [57]:
tol_induced_subtree(c(292466, 267845, 316878, 102710), file = "test2")

[1] TRUE